In [2]:
import re
import getpass
import pandas as pd
import os
import pydicom
from pydicom.tag import Tag
import datetime
import glob
import math
import csv
from redcap import Project, RedcapError

In [3]:
# connect to redcap (note: exporting means retrieving from redcap; importing means uploading to redcap)

api_url = 'https://redcap.partners.org/redcap/api/'
api_key = '9D668EE32160684D980CD6A95B23CC5B'
# to set specific certificate pathway: verify_ssl='path/to/CA_bundle/' or set to False to ignore SSH verification altogether
project = Project(api_url, api_key) # pycap verifies SSL certificate under the hood (using requests)
# will return a list of dicts from all record IDs with all raw field names as keys
data = project.export_records(format='df')

In [4]:
data

,redcap_repeat_instrument,redcap_repeat_instance,site_name,sex,group,notes,scan_date,session_id,age,download_date,...,rh_superiorparietal_thickness,rh_superiortemporal_thickness,rh_supramarginal_thickness,rh_frontalpole_thickness,rh_temporalpole_thickness,rh_transversetemporal_thickness,rh_insula_thickness,rh_meanthickness_thickness,rh_brainsegvolnotvent,rh_etiv
record_id,,,,,,,,,,,,,,,,,,,,,
LDS0110021,NaN,NaN,WashUSL,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LDS0110021,visits,1.0,NaN,NaN,NaN,NaN,2018-10-16,LDS0110021_20181016,59.0,NaN,...,1.867,2.503,2.015,2.275,3.015,2.580,2.488,2.14189,878094.0,1.467879e+06
LDS0110022,NaN,NaN,WashUSL,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LDS0110022,visits,1.0,NaN,NaN,NaN,NaN,2018-11-29,LDS0110022_20181129,60.0,NaN,...,2.037,2.757,2.442,2.455,3.660,2.464,2.854,2.37360,954774.0,1.403224e+06
LDS0110040,NaN,NaN,WashUSL,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LDS0110040,visits,1.0,NaN,NaN,NaN,NaN,2019-01-24,LDS0110040_20190124,60.0,NaN,...,2.122,2.616,2.236,2.491,3.741,2.297,2.815,2.28576,967881.0,1.422998e+06
LDS0220026,NaN,NaN,UPenn,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LDS0220026,visits,1.0,NaN,NaN,NaN,NaN,2018-11-09,LDS0220026_20181109,118.0,NaN,...,2.247,2.842,2.557,2.615,3.547,2.506,3.100,2.44725,1075509.0,1.522366e+06
LDS0220031,NaN,NaN,UPenn,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [307]:
### API FOR MASTER!!!! 

api_url = 'https://redcap.partners.org/redcap/api/'
api_key = '49E6305CE4B41073E2342F723C0F93F8'
project = Project(api_url, api_key) 

# look into how to print all sessions of a certain subject

#Form Name = scan_session
#Variable / Field Name = scan_date , recon1_fsversion , recon1_qc_path, scan_mr_resolution

# to_export = [{'record':'JPIE_M_37','form':'scan_session'}]
# data = project.export_records(to_export)

ids_of_interest =['SMAT_F_64']
#subset = project.export_records(records=ids_of_interest)
# Contains all fields, but only three records
# Known fields of interest
fields_of_interest =['scan_date','scan_type','scan_project_id','recon1_qc_status','recon1_username','scan_mr_resolution','recon1_quality','recon1_fsversion','recon1_qc_path','recon1_quality','recon2_state','recon2_fsversion','recon2_qc_path','recon2_quality','recon3_qc_path']
subset = project.export_records(records=ids_of_interest, fields=fields_of_interest)

In [308]:
subset
# count = 0
# for s in subset:
#     count = count + 1
#     print(count)
#     print(s.get('recon2_qc_path'))

[{'subject_id': 'SMAT_F_64',
  'redcap_event_name': 'static_arm_1',
  'scan_date': '2019-03-28',
  'scan_project_id': 'LEADS',
  'scan_type': 'PET',
  'scan_mr_resolution': '',
  'recon1_qc_path': '',
  'recon1_fsversion': '',
  'recon1_quality': '',
  'recon1_qc_status': '',
  'recon1_username': '',
  'recon2_qc_path': '',
  'recon2_fsversion': '',
  'recon2_quality': '',
  'recon3_qc_path': ''},
 {'subject_id': 'SMAT_F_64',
  'redcap_event_name': 'visit_2_arm_1',
  'scan_date': '2019-08-06',
  'scan_project_id': 'LEADS',
  'scan_type': 'MR',
  'scan_mr_resolution': '',
  'recon1_qc_path': '',
  'recon1_fsversion': '',
  'recon1_quality': '',
  'recon1_qc_status': '',
  'recon1_username': '',
  'recon2_qc_path': '',
  'recon2_fsversion': '',
  'recon2_quality': '',
  'recon3_qc_path': ''},
 {'subject_id': 'SMAT_F_64',
  'redcap_event_name': 'visit_3_arm_1',
  'scan_date': '2019-08-20',
  'scan_project_id': 'LEADS',
  'scan_type': 'PET',
  'scan_mr_resolution': '',
  'recon1_qc_path': 

In [255]:
session_of_interest = project.export_records(events=['visit_5_arm_1'],records=['MRIC_F_46'],fields=['recon1_username','recon1_qc_status'],format='df')
#session_of_interest = project.export_records(records=['DTUB_M_48'],events=['visit_11_arm_1'],forms=['scan_session'],format='df')

In [256]:
session_edit = session_of_interest
session_of_interest

,,recon1_qc_status,recon1_username
subject_id,redcap_event_name,,
MRIC_F_46,visit_5_arm_1,NaN,NaN


In [257]:
session_edit.loc[:,'recon1_username'] = 'RJE'
session_edit.loc[:,'recon1_qc_status'] = 'inprog'
# or [{'A':0,'C':0,'G':0,'T':0},{'A':0,'C':0,'G':0,'T':0},{'A':0,'C':0,'G':0,'T':0},...]

In [213]:
pd.set_option('display.max_columns', None)
session_edit

,,recon1_qc_status,recon1_username
subject_id,redcap_event_name,,
DTUB_M_48,visit_11_arm_1,inprog,RJE


In [221]:
session_of_interest

,,recon1_qc_status,recon1_username
subject_id,redcap_event_name,,
DTUB_M_48,visit_11_arm_1,inprog,RJE


In [258]:
############# IMPORTING STATUS INPROG == USERNAME REZZO

response = project.import_records(session_edit, date_format='YMD',format='df') 